<a href="https://colab.research.google.com/github/Breysel/HexAI/blob/danigoeswild/hexAIputer.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [3]:
print('testtestt')

testtest
